# Vergleich von Imputation Methoden

An dieser Stelle sollen verschiede Methoden zum interpolieren von fehlenden Werten betrachtet und verglichen werden.

### Vorbereitung

In [1]:
import pandas as pd
import numpy as np
import math
import time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

In [2]:
results = []

In [3]:
def reset_base():
    base= pd.read_csv('additional_data/base.csv') 
    base.set_index(['Country Name', 'Indicator Name'], inplace=True)
    base = base.sort_index(level=['Country Name', 'Indicator Name'])
    return base
base = reset_base()

def get_cords(frac, rnd_state):
    n = int(base.isna().sum().sum()*frac)
    print(f'Testdaten mit {frac*100}% fehlenden Werten (absolut: {n})')
    #random state to ensure reproducibility
    rnds = np.random.RandomState(rnd_state)

    #coordinates for data entries to be removed randomly
    #5000 entries are selected
    cords = pd.DataFrame([[rnds.randint(0, len(base), size=n*4)[i], 
                  rnds.randint(0, len(base.columns), size=n*4)[i]]
                  for i in range(n*4)])

    #all coordinates pointing to NaN entries are removed and
    #first 1000 remaining entries are selected
    cords['value'] = [base.iloc[cords[0][i], cords[1][i]] for i in cords.index]
    cords = cords.dropna()[:n].reset_index(drop=True)
    
    return cords

def reset_train(cords):
    train = reset_base()
    for i in cords.index:
        train.iloc[cords[0][i], cords[1][i]] = None
    return train

def evaluate(df, t, cords):
    
    
    #scaling original data and imputed data
    #necessary ?????????????????????????????????????
    train = reset_train(cords)
    scaler = StandardScaler().fit(train) #fitting on train?
    norm_base = pd.DataFrame(scaler.transform(base))
    df = pd.DataFrame(scaler.transform(df))

    #getting imputed values for simulated NaNs and true value 
    res =pd.DataFrame({'y_true': [norm_base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                       'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index]
                      })
    res = res.dropna()

   
    #calculate evaluation metrics
    r2 = r2_score(res['y_true'], res['y_pred'])
    rmse = math.sqrt(mean_squared_error(res['y_true'], res['y_pred']))
    still_missing = df.isna().sum().sum()
    
    print(f'Mit dieser Methode bleiben {still_missing} NaNs bestehen.')
    print(f'{len(res)} Werte wurden für die Metriken verwendet.')
    print(f'r2: {r2}, rmse: {rmse}')
    print('')
    
    return [r2, rmse, still_missing, t]



In [4]:
def impute_backfill(df):
    df = df.fillna(method='bfill', limit=3)
    return df

def impute_overall_means(df):
    #fill NaNs with overall mean of that indicator
    values = pd.DataFrame(df.stack()).groupby('Indicator Name')[0].mean()
    df = pd.DataFrame(df.stack(dropna=False))
    
    df[0] = df[0].fillna(df.groupby('Indicator Name')[0].transform('mean'))
    df = df.unstack()
    df.columns = df.columns.droplevel(0)
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
        
    return df

def impute_yearly_means(df):
    #fill NaNs with overall mean of that indicator
    
    for i in df.columns:
        df[i] = df[i].fillna(df.groupby('Indicator Name')[i].transform('mean'))
            
    return df

def impute_yearly_means_per_region(df):
    country_data = pd.read_csv('../Data/WDICountry.csv')
    country_data = country_data.loc[:,['Table Name', 'Region']]
    df = pd.merge(df.reset_index(), country_data, how='left', left_on='Country Name', right_on='Table Name').drop('Table Name', axis=1)
    df = df.set_index(['Country Name', 'Indicator Name', 'Region'])

    for i in df.columns:
        df[i] = df[i].fillna(df.groupby(['Indicator Name', 'Region'])[i].transform('mean'))

    df = df.reset_index().set_index(['Country Name', 'Indicator Name']).drop('Region', axis=1)
    return df

def interpolate3(df):
    df = df.interpolate(limit=3)
    return df

def interpolate_all(df):
    df = df.interpolate()
    return df

def iterative_imputer_1(df):
    col = df.columns
    idx = df.index
    
    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)
    df= pd.DataFrame(df, columns=col, index=idx)
    return df

def iterative_imputer_2(df):
    df = df.unstack().T
    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

def iterative_imputer_3(df):

    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999, verbose=True)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

def mice_imputer(df):
    n_imputations =  12
    dfs = []
    col = df.columns
    idx = df.index
    
    for i in range(n_imputations): 
        print(f'Imputation round {i}')
        iter_imp = IterativeImputer(random_state=i, sample_posterior=True, verbose=2)
        df_temp = iter_imp.fit_transform(df)
        dfs.append(df_temp)
    
    df = np.mean(np.array(dfs), axis=0)
    df = pd.DataFrame(df, columns=col, index=idx)
    return df

def knn_imputer1(df):
    col = df.columns
    idx = df.index
    
    knn_imp = KNNImputer(n_neighbors=2)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)
    return df

def knn_imputer2(df, n=4):
    
    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    knn_imp = KNNImputer(n_neighbors=n)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)

    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

In [5]:


functions =  {'Backfill':impute_backfill, 'Overall Mean':impute_overall_means, 
              'Yearly Mean':impute_yearly_means, 'Yearly Mean per Region':impute_yearly_means_per_region, 
              'Interpolate 3':interpolate3, 'Interpolate all':interpolate_all, 
              'ICE 1':iterative_imputer_1, 'ICE 2':iterative_imputer_2, 
              'ICE3':iterative_imputer_3, 'MICE':mice_imputer, 
              'KNN 1':knn_imputer1, 'KNN 2':knn_imputer2
             }



In [6]:
def run_all(frac):
    
    cords_list =[]
    for i in np.arange(1,6):
        cords_list.append(get_cords(frac, i))
    
    results = {}
    detailed_results = {}    
    
    for name, func in functions.items():
        func_runs = []
        print(name)
        
        for cords in cords_list:
            
            t0 = time.time()
            df= func(reset_train(cords)) 
            t1 = time.time()

            t = t1-t0

            func_runs.append(evaluate(df, t, cords))
        
        print(func_runs)
        detailed_results[name] = func_runs
        results[name] = np.mean(np.array(func_runs), axis=0)
        
    return results, detailed_results
r1, r1_detail = run_all(0.05)

Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Testdaten mit 5.0% fehlenden Werten (absolut: 8515)
Backfill
Mit dieser Methode bleiben 32924 NaNs bestehen.

8195 Werte wurden für die Metriken verwendet.
r2: -0.2630386067986965, rmse: 3.1264580984032304
Mit dieser Methode bleiben 32825 NaNs bestehen.

8263 Werte wurden für die Metriken verwendet.
r2: -2.77533240911633, rmse: 1.1896157293266358
Mit dieser Methode bleiben 32824 NaNs bestehen.

8250 Werte wurden für die Metriken verwendet.
r2: -1.688280205196218, rmse: 1.0226797653748734
Mit dieser Methode bleiben 32841 NaNs bestehen.

8249 Werte wurden für die Metriken verwendet.
r2: -0.6672076075877178, rmse: 2.085863590967128
Mit dieser Methode bleiben 32828 NaNs bestehen.

8240 Werte wurden für die Metriken verwendet.
r2: -0.07720522781720884, rmse: 3.533882957257935
[[-0.26303

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9942158016960598, rmse: 0.20756410705991854


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.7827329807258437, rmse: 0.2820413273792655


...

[IterativeImputer] Ending imputation round 3/10, elapsed time 3.41
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.56
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.77
[IterativeImputer] Ending imputation round 6/10, elapsed time 7.01
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.14
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.28
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.40
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.57
Imputation round 11
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.29
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.44
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.59
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.69
[IterativeImputer] Ending imputation round 6/1

[IterativeImputer] Ending imputation round 1/10, elapsed time 1.13
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.26
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.38
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.82
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.96
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.10
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.25
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.37
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.55
Imputation round 10
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.27
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.40
[IterativeImputer] Ending imputation round 4/1

[IterativeImputer] Ending imputation round 9/10, elapsed time 10.34
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.49
Imputation round 8
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.32
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.52
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.72
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.82
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.98
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.10
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.28
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.41
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.54
Imputation round 9
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputa

[IterativeImputer] Ending imputation round 6/10, elapsed time 6.95
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.10
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.30
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.48
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.74
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.30
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.51
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.70
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.95
[IterativeImputer] Ending imputation round 5/10, elapsed time 6.13
[IterativeImputer] Ending imputation round 6/10, elapsed time 7.32
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.54
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.72
[IterativeImputer] Ending imputation round 9/10

[IterativeImputer] Ending imputation round 3/10, elapsed time 3.45
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.59
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.75
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.90
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.06
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.24
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.48
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.64
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.15
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.29
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.43
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.56
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.71
[IterativeImputer] Ending imputation round 6/10

In [18]:
r1 = pd.DataFrame(r1).T
r1

,0,1,2,3
Backfill,-1.094213,2.191700,32848.4,0.855777
Overall Mean,-0.251541,1.805936,0.0,1.097199
Yearly Mean,0.009964,1.768969,52298.0,0.941444
Yearly Mean per Region,0.016326,1.770021,58352.6,1.063025
Interpolate 3,-0.557857,1.999203,32879.0,0.924791
Interpolate all,-0.557853,1.966919,58.0,0.890814
ICE 1,0.948744,0.210681,0.0,10.586561
ICE 2,0.926469,0.448741,0.0,24.251318
ICE3,0.703721,0.927913,0.0,85.768895
MICE,0.951523,0.236147,0.0,140.065102


In [12]:
r1_detail = pd.DataFrame(pd.DataFrame(r1_detail).T.stack().to_dict()).T
r1_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r1_detail

r2      rmse  still_missing           t
Backfill               0 -0.263039  3.126458        32924.0    0.823884
                       1 -2.775332  1.189616        32825.0    0.856179
                       2 -1.688280  1.022680        32824.0    0.919926
                       3 -0.667208  2.085864        32841.0    0.826850
                       4 -0.077205  3.533883        32828.0    0.852047
Overall Mean           0  0.056692  2.650681            0.0    1.077665
                       1 -0.790210  0.809595            0.0    1.164970
                       2 -0.553206  0.765509            0.0    1.078683
                       3 -0.089972  1.660006            0.0    1.066062
                       4  0.118993  3.143888            0.0    1.098613
Yearly Mean            0  0.008985  2.716882        52298.0    0.927185
                       1 -0.003552  0.606158        52298.0    0.936965
                       2  0.025439  0.606373        52298.0    0.934207
                       3  0.008787  1.583016        52298.0    0.923039
                       4  0.010162  3.332418        52298.0    0.985821
Yearly Mean per Region 0  0.043249  2.697510        58365.0    1.069834
                       1 -0.133955  0.651156        58318.0    1.079807
                       2  0.142235  0.575184        58345.0    1.052537
                       3 -0.016975  1.620865        58359.0    1.047099
                       4  0.047073  3.305390        58376.0    1.065846
Interpolate 3          0 -0.072788  2.873685        32955.0    0.900497
                       1 -1.525109  0.977054        32855.0    0.950032
                       2 -0.930706  0.867540        32855.0    0.938470
                       3 -0.198324  1.769570        32872.0    0.910423
                       4 -0.062355  3.508164        32858.0    0.924532
Interpolate all        0 -0.072774  2.826733           58.0    0.864921
                       1 -1.525022  0.961498           58.0    0.885435
                       2 -0.930811  0.853504           58.0    0.874557
                       3 -0.198316  1.740554           58.0    0.934982
                       4 -0.062341  3.452306           58.0    0.894177
ICE 1                  0  0.994216  0.207564            0.0   11.710053
                       1  0.782733  0.282041            0.0   10.201523
                       2  0.988237  0.066620            0.0   10.328932
                       3  0.987720  0.176199            0.0   10.431111
                       4  0.990817  0.320980            0.0   10.261185
ICE 2                  0  0.897000  0.875889            0.0   24.361761
                       1  0.966479  0.110783            0.0   24.900007
                       2  0.850137  0.237784            0.0   23.805398
                       3  0.965368  0.295899            0.0   24.211332
                       4  0.953361  0.723353            0.0   23.978091
ICE3                   0  0.672465  1.561922            0.0   85.835097
                       1  0.864944  0.222368            0.0   85.706950
                       2  0.436893  0.460926            0.0   84.841991
                       3  0.793355  0.722794            0.0   85.688769
                       4  0.750950  1.671555            0.0   86.771667
MICE                   0  0.992594  0.234863            0.0  140.250843
                       1  0.815449  0.259941            0.0  139.079629
                       2  0.993037  0.051253            0.0  139.651592
                       3  0.967271  0.287652            0.0  141.360313
                       4  0.989266  0.347026            0.0  139.983132
KNN 1                  0  0.549804  1.831182            0.0  102.488866
                       1  0.155114  0.556180            0.0  102.214341
                       2  0.358112  0.492113            0.0  102.729743
                       3  0.018670  1.575105            0.0  101.131126
                       4  0.282422  2.837343            0.0  100.992824


In [8]:
r2 , r2_detail =  run_all(0.075)

Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Testdaten mit 7.5% fehlenden Werten (absolut: 12773)
Backfill
Mit dieser Methode bleiben 33252 NaNs bestehen.

12384 Werte wurden für die Metriken verwendet.
r2: -0.7622360204251646, rmse: 1.9101575370444133
Mit dieser Methode bleiben 33280 NaNs bestehen.

12322 Werte wurden für die Metriken verwendet.
r2: -0.13326755926364164, rmse: 1.5732663625326984
Mit dieser Methode bleiben 33292 NaNs bestehen.

12334 Werte wurden für die Metriken verwendet.
r2: -0.15471382925960686, rmse: 2.34729184379117
Mit dieser Methode bleiben 33312 NaNs bestehen.

12333 Werte wurden für die Metriken verwendet.
r2: -0.38762500614250417, rmse: 2.1233606768359277
Mit dieser Methode bleiben 33191 NaNs bestehen.

12365 Werte wurden für die Metriken verwendet.
r2: -0.12638032635757868, rmse: 1.86746347988

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

12773 Werte wurden für die Metriken verwendet.
r2: 0.9803957096839419, rmse: 0.19838139998923512


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

12773 Werte wurden für die Metriken verwendet.
r2: 0.9755494244776735, rmse: 0.22712199683626336


...

[IterativeImputer] Ending imputation round 3/10, elapsed time 3.58
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.82
[IterativeImputer] Ending imputation round 5/10, elapsed time 6.15
[IterativeImputer] Ending imputation round 6/10, elapsed time 7.53
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.92
[IterativeImputer] Ending imputation round 8/10, elapsed time 10.42
[IterativeImputer] Ending imputation round 9/10, elapsed time 11.74
[IterativeImputer] Ending imputation round 10/10, elapsed time 12.89
Imputation round 11
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.13
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.26
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.39
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.54
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.70
[IterativeImputer] Ending imputation round 6/

[IterativeImputer] Ending imputation round 1/10, elapsed time 1.15
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.31
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.47
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.60
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.80
[IterativeImputer] Ending imputation round 6/10, elapsed time 7.05
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.25
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.36
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.49
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.61
Imputation round 10
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.13
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.23
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.37
[IterativeImputer] Ending imputation round 4/1

[IterativeImputer] Ending imputation round 9/10, elapsed time 10.35
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.47
Imputation round 8
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.15
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.25
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.43
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.68
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.85
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.98
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.12
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.23
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.41
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.59
Imputation round 9
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputa

[IterativeImputer] Ending imputation round 6/10, elapsed time 6.85
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.01
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.15
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.33
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.55
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.22
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.32
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.47
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.62
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.76
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.87
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.03
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.15
[IterativeImputer] Ending imputation round 9/10

[IterativeImputer] Ending imputation round 3/10, elapsed time 3.37
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.48
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.64
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.79
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.01
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.26
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.38
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.51
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.10
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.28
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.42
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.59
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.70
[IterativeImputer] Ending imputation round 6/10

In [19]:
r2 = pd.DataFrame(r2).T
r2

,0,1,2,3
Backfill,-0.312845,1.964308,33265.4,1.128651
Overall Mean,-0.052291,1.740381,0.0,1.382833
Yearly Mean,0.014764,1.692044,52298.0,1.204775
Yearly Mean per Region,0.049003,1.681218,58425.0,1.320798
Interpolate 3,-0.219687,1.909891,33296.8,1.190733
Interpolate all,-0.221227,1.880009,59.6,1.167769
ICE 1,0.986040,0.182850,0.0,10.338374
ICE 2,0.945163,0.365983,0.0,26.355917
ICE3,0.785148,0.771223,0.0,89.383753
MICE,0.988788,0.168885,0.0,141.106209


In [13]:
r2_detail = pd.DataFrame(pd.DataFrame(r3_detail).T.stack().to_dict()).T
r2_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r2_detail

r2      rmse  still_missing           t
Backfill               0 -3.964220  1.152188        33619.0    1.433451
                       1 -1.837371  1.679247        33786.0    1.440451
                       2 -1.908139  1.523993        33654.0    1.446450
                       3 -0.897788  1.538364        33643.0    1.433474
                       4 -1.176092  1.413485        33679.0    1.445665
Overall Mean           0 -1.083659  0.734942            0.0    1.674192
                       1 -0.203727  1.073588            0.0    1.738185
                       2 -0.364231  1.025791            0.0    1.666184
                       3 -0.178540  1.191847            0.0    1.675212
                       4 -0.280922  1.066565            0.0    1.658654
Yearly Mean            0 -0.037370  0.518569        52298.0    1.530683
                       1  0.015803  0.970766        52298.0    1.497994
                       2  0.003029  0.876938        52456.0    1.527034
                       3  0.018013  1.087931        52298.0    1.527044
                       4  0.018624  0.933563        52298.0    1.515010
Yearly Mean per Region 0 -0.373088  0.603081        58608.0    1.649652
                       1  0.051644  0.963471        58578.0    1.604604
                       2  0.015290  0.881940        58645.0    1.619635
                       3 -0.026592  1.123984        58586.0    1.635181
                       4 -0.006983  0.956185        58734.0    1.607591
Interpolate 3          0 -2.091582  0.910245        33649.0    1.489898
                       1 -0.559111  1.242379        33816.0    1.531580
                       2 -0.762802  1.185136        33686.0    1.518024
                       3 -0.426421  1.333826        33674.0    1.539076
                       4 -0.615634  1.218606        33709.0    1.519040
Interpolate all        0 -2.091558  0.895217           58.0    1.531235
                       1 -0.559080  1.221822           58.0    1.477851
                       2 -0.762587  1.166012           59.0    1.480222
                       3 -0.434792  1.315052           58.0    1.502025
                       4 -0.615608  1.197862           59.0    1.535557
ICE 1                  0  0.937020  0.127773            0.0   10.421709
                       1  0.952143  0.214066            0.0   10.416087
                       2  0.989395  0.090442            0.0   10.494919
                       3  0.987255  0.123943            0.0   10.544736
                       4  0.914507  0.275545            0.0   10.655497
ICE 2                  0  0.935392  0.129414            0.0   24.363670
                       1  0.975435  0.153368            0.0   24.487075
                       2  0.936051  0.222092            0.0   24.599819
                       3  0.978803  0.159841            0.0   24.520411
                       4  0.965416  0.175252            0.0   24.340196
ICE3                   0  0.865651  0.186619            0.0   86.827502
                       1  0.787376  0.451211            0.0   86.827588
                       2  0.680454  0.496457            0.0   88.225254
                       3  0.479221  0.792274            0.0   87.288222
                       4  0.872251  0.336826            0.0   87.298270
MICE                   0  0.961929  0.099342            0.0  140.174898
                       1  0.955398  0.206657            0.0  138.988162
                       2  0.989838  0.088533            0.0  139.444724
                       3  0.992163  0.097189            0.0  143.873310
                       4  0.940582  0.229712            0.0  139.979002
KNN 1                  0  0.135463  0.473404            0.0  105.349567
                       1  0.161437  0.896069            0.0  103.510660
                       2  0.515733  0.611164            0.0  103.528869
                       3  0.145595  1.014800            0.0  103.657478
                       4  0.202876  0.841374            0.0  104.345485


In [10]:
r3, r3_detail = run_all(0.1)

Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Testdaten mit 10.0% fehlenden Werten (absolut: 17030)
Backfill
Mit dieser Methode bleiben 33619 NaNs bestehen.

16464 Werte wurden für die Metriken verwendet.
r2: -3.964219549140708, rmse: 1.1521878884113992
Mit dieser Methode bleiben 33786 NaNs bestehen.

16407 Werte wurden für die Metriken verwendet.
r2: -1.8373707535864638, rmse: 1.6792466658151293
Mit dieser Methode bleiben 33654 NaNs bestehen.

16447 Werte wurden für die Metriken verwendet.
r2: -1.9081389310564236, rmse: 1.5239926108614468
Mit dieser Methode bleiben 33643 NaNs bestehen.

16460 Werte wurden für die Metriken verwendet.
r2: -0.8977876217879388, rmse: 1.5383638773729555
Mit dieser Methode bleiben 33679 NaNs bestehen.

16467 Werte wurden für die Metriken verwendet.
r2: -1.1760918948068961, rmse: 1.413484968

C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

17030 Werte wurden für die Metriken verwendet.
r2: 0.9370202334659707, rmse: 0.12777336992410396


C:\Users\schwa\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

17030 Werte wurden für die Metriken verwendet.
r2: 0.9521427024799756, rmse: 0.2140660895293748


...

[IterativeImputer] Ending imputation round 3/10, elapsed time 3.36
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.50
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.62
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.73
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.85
[IterativeImputer] Ending imputation round 8/10, elapsed time 8.98
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.07
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.25
Imputation round 11
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.17
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.28
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.35
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.48
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.55
[IterativeImputer] Ending imputation round 6/1

[IterativeImputer] Ending imputation round 1/10, elapsed time 1.14
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.24
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.37
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.47
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.61
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.72
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.86
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.07
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.31
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.42
Imputation round 10
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.12
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.24
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.40
[IterativeImputer] Ending imputation round 4/1

[IterativeImputer] Ending imputation round 9/10, elapsed time 10.28
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.40
Imputation round 8
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.19
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.34
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.53
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.65
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.82
[IterativeImputer] Ending imputation round 6/10, elapsed time 7.00
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.28
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.51
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.68
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.84
Imputation round 9
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputa

[IterativeImputer] Ending imputation round 6/10, elapsed time 7.28
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.47
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.63
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.78
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.96
Imputation round 7
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.20
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.44
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.67
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.81
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.97
[IterativeImputer] Ending imputation round 6/10, elapsed time 7.10
[IterativeImputer] Ending imputation round 7/10, elapsed time 8.32
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.50
[IterativeImputer] Ending imputation round 9/10

[IterativeImputer] Ending imputation round 3/10, elapsed time 3.27
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.38
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.57
[IterativeImputer] Ending imputation round 6/10, elapsed time 6.81
[IterativeImputer] Ending imputation round 7/10, elapsed time 7.92
[IterativeImputer] Ending imputation round 8/10, elapsed time 9.05
[IterativeImputer] Ending imputation round 9/10, elapsed time 10.13
[IterativeImputer] Ending imputation round 10/10, elapsed time 11.24
Imputation round 6
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.06
[IterativeImputer] Ending imputation round 2/10, elapsed time 2.17
[IterativeImputer] Ending imputation round 3/10, elapsed time 3.27
[IterativeImputer] Ending imputation round 4/10, elapsed time 4.38
[IterativeImputer] Ending imputation round 5/10, elapsed time 5.49
[IterativeImputer] Ending imputation round 6/10

In [20]:
r3 = pd.DataFrame(r3).T
r3

,0,1,2,3
Backfill,-1.956722,1.461455,33676.2,1.439898
Overall Mean,-0.422216,1.018547,0.0,1.682485
Yearly Mean,0.003620,0.877554,52329.6,1.519553
Yearly Mean per Region,-0.067946,0.905732,58630.2,1.623332
Interpolate 3,-0.891110,1.178038,33706.8,1.519524
Interpolate all,-0.892725,1.159193,58.4,1.505378
ICE 1,0.956064,0.166354,0.0,10.506589
ICE 2,0.958219,0.167993,0.0,24.462234
ICE3,0.736991,0.452677,0.0,87.293367
MICE,0.967982,0.144287,0.0,140.492019


In [14]:
r3_detail = pd.DataFrame(pd.DataFrame(r3_detail).T.stack().to_dict()).T
r3_detail.columns = ['r2', 'rmse', 'still_missing', 't']
r3_detail

r2      rmse  still_missing           t
Backfill               0 -3.964220  1.152188        33619.0    1.433451
                       1 -1.837371  1.679247        33786.0    1.440451
                       2 -1.908139  1.523993        33654.0    1.446450
                       3 -0.897788  1.538364        33643.0    1.433474
                       4 -1.176092  1.413485        33679.0    1.445665
Overall Mean           0 -1.083659  0.734942            0.0    1.674192
                       1 -0.203727  1.073588            0.0    1.738185
                       2 -0.364231  1.025791            0.0    1.666184
                       3 -0.178540  1.191847            0.0    1.675212
                       4 -0.280922  1.066565            0.0    1.658654
Yearly Mean            0 -0.037370  0.518569        52298.0    1.530683
                       1  0.015803  0.970766        52298.0    1.497994
                       2  0.003029  0.876938        52456.0    1.527034
                       3  0.018013  1.087931        52298.0    1.527044
                       4  0.018624  0.933563        52298.0    1.515010
Yearly Mean per Region 0 -0.373088  0.603081        58608.0    1.649652
                       1  0.051644  0.963471        58578.0    1.604604
                       2  0.015290  0.881940        58645.0    1.619635
                       3 -0.026592  1.123984        58586.0    1.635181
                       4 -0.006983  0.956185        58734.0    1.607591
Interpolate 3          0 -2.091582  0.910245        33649.0    1.489898
                       1 -0.559111  1.242379        33816.0    1.531580
                       2 -0.762802  1.185136        33686.0    1.518024
                       3 -0.426421  1.333826        33674.0    1.539076
                       4 -0.615634  1.218606        33709.0    1.519040
Interpolate all        0 -2.091558  0.895217           58.0    1.531235
                       1 -0.559080  1.221822           58.0    1.477851
                       2 -0.762587  1.166012           59.0    1.480222
                       3 -0.434792  1.315052           58.0    1.502025
                       4 -0.615608  1.197862           59.0    1.535557
ICE 1                  0  0.937020  0.127773            0.0   10.421709
                       1  0.952143  0.214066            0.0   10.416087
                       2  0.989395  0.090442            0.0   10.494919
                       3  0.987255  0.123943            0.0   10.544736
                       4  0.914507  0.275545            0.0   10.655497
ICE 2                  0  0.935392  0.129414            0.0   24.363670
                       1  0.975435  0.153368            0.0   24.487075
                       2  0.936051  0.222092            0.0   24.599819
                       3  0.978803  0.159841            0.0   24.520411
                       4  0.965416  0.175252            0.0   24.340196
ICE3                   0  0.865651  0.186619            0.0   86.827502
                       1  0.787376  0.451211            0.0   86.827588
                       2  0.680454  0.496457            0.0   88.225254
                       3  0.479221  0.792274            0.0   87.288222
                       4  0.872251  0.336826            0.0   87.298270
MICE                   0  0.961929  0.099342            0.0  140.174898
                       1  0.955398  0.206657            0.0  138.988162
                       2  0.989838  0.088533            0.0  139.444724
                       3  0.992163  0.097189            0.0  143.873310
                       4  0.940582  0.229712            0.0  139.979002
KNN 1                  0  0.135463  0.473404            0.0  105.349567
                       1  0.161437  0.896069            0.0  103.510660
                       2  0.515733  0.611164            0.0  103.528869
                       3  0.145595  1.014800            0.0  103.657478
                       4  0.202876  0.841374            0.0  104.345485


In [21]:
sets = {'r1':r1, 'r1_detail':r1_detail,
       'r2':r2, 'r2_detail':r2_detail,
       'r3':r3, 'r3_detail':r3_detail,}
for name, df in sets.items():
    df.to_csv(f'additional_data/results/{name}.csv')